In [1]:
import os
import sys

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
s3_conf = {
    "spark.hadoop.fs.s3a.endpoint":          "http://minio:9000",
    "spark.hadoop.fs.s3a.impl":              "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.access.key":        "admin",
    "spark.hadoop.fs.s3a.secret.key":        "SuperSecr3t",
    "spark.hadoop.fs.s3a.path.style.access": True,
}

In [3]:
spark_version = pyspark.version.__version__
packages = [
    f"org.apache.spark:spark-avro_2.11:{spark_version}",
]

In [4]:
conf = SparkConf()
conf.setAll(s3_conf.items())
conf.set("spark.jars.packages", ",".join(packages))

In [5]:
spark = SparkSession.builder\
        .appName("Avro sample")\
        .config(conf=conf)\
        .getOrCreate()

# Read Arvo file

In [6]:
df = spark.read.format("avro").load("s3a://kafka/topics/sample_mysql.sakila.actor")

In [7]:
df.show(truncate=False)

+--------+----------+-----------+--------------------+---------+
|actor_id|first_name|last_name  |last_update         |partition|
+--------+----------+-----------+--------------------+---------+
|111     |CAMERON   |ZELLWEGER  |2006-02-15T04:34:33Z|0        |
|112     |RUSSELL   |BACALL     |2006-02-15T04:34:33Z|0        |
|113     |MORGAN    |HOPKINS    |2006-02-15T04:34:33Z|0        |
|114     |MORGAN    |MCDORMAND  |2006-02-15T04:34:33Z|0        |
|115     |HARRISON  |BALE       |2006-02-15T04:34:33Z|0        |
|116     |DAN       |STREEP     |2006-02-15T04:34:33Z|0        |
|117     |RENEE     |TRACY      |2006-02-15T04:34:33Z|0        |
|118     |CUBA      |ALLEN      |2006-02-15T04:34:33Z|0        |
|119     |WARREN    |JACKMAN    |2006-02-15T04:34:33Z|0        |
|120     |PENELOPE  |MONROE     |2006-02-15T04:34:33Z|0        |
|61      |CHRISTIAN |NEESON     |2006-02-15T04:34:33Z|0        |
|62      |JAYNE     |NEESON     |2006-02-15T04:34:33Z|0        |
|63      |CAMERON   |WRAY

In [8]:
df = df.coalesce(1)

In [7]:
df.write.format("avro").save("s3a://raw/sakila/actor", mode="overwrite")

In [8]:
spark